In [2]:
!pip install opencv-python
!pip install matplotlib

In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [42]:
cap = cv2.VideoCapture('Exercise1Files/Traffic_Laramie_1.mp4')

# Get the frame rate of the video
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the region of interest (ROI)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
roi_x = 0
roi_y = int(height / 2)
roi_w = width
roi_h = int(height / 2)
x1, y1 = roi_x, roi_y
x2, y2 = roi_x + roi_w, roi_y + roi_h
text = 'Main Street'

# Define the background subtractor
fgbg = cv2.createBackgroundSubtractorMOG2()

# Define minimum area threshold
min_area = 500

# Create red rectangle with text
mainstreet = np.zeros((height, width, 3), dtype=np.uint8)
cv2.rectangle(mainstreet, (x1, y1), (x2, y2), (0, 0, 255), thickness=2, lineType=cv2.LINE_AA)
cv2.rectangle(mainstreet, (x1+2, y1+2), (x2-2, y2-2), (0, 0, 0), thickness=-1, lineType=cv2.LINE_AA)
cv2.putText(mainstreet, text, (x1+10, y1+25), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)


array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[  0,   0, 255],
        [  0,   0, 202],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0, 197]],

       [[  0,   0, 255],
        [  0,   0, 228],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0, 222]],

       [[  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 224],
        ...,
        [  0,   0, 197],
        [  0,   0, 222],
        [  0,   0, 255]]

In [43]:
# Process each frame and detect cars
while True:
    # Read a frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Apply the ROI to the frame
    frame_roi = frame[roi_y:roi_y+roi_h, roi_x:roi_x+roi_w]
    
    # Apply the background subtractor to the ROI
    fgmask = fgbg.apply(frame_roi)
    
    # Binarize difference image
    th = cv2.threshold(fgmask.copy(), 200, 255, cv2.THRESH_BINARY)[1]
    
    # Apply morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    th = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel)
    
     # Find contours
    contours, hierarchy = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        # Filter out small contours
        if cv2.contourArea(c) < min_area:
            continue

        # Calculate the bounding box of the contour
        (x, y, w, h) = cv2.boundingRect(c)

        # Filter out non-rectangular contours
        aspect_ratio = w / float(h)
        if aspect_ratio < 0.5:
            continue

        # Draw bounding box around moving object
        cv2.rectangle(frame, (x+roi_x, y+roi_y), (x+roi_x+w, y+roi_y+h), (0, 255, 0), 2)

    # Add main street rectangle to the frame
    frame += mainstreet 
    
    alpha = 0.5
    beta = 1 - alpha
    blended = cv2.addWeighted(frame, alpha, mainstreet, beta, 0)
    
    # Convert blended image to HSV color space
    hsv = cv2.cvtColor(blended, cv2.COLOR_BGR2HSV)

    # Increase value channel
    hsv[:,:,2] += 30

    # Convert back to BGR color space
    result = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
    # Display output
    cv2.imshow('frame', result)

    # Exit program when 'q' key is pressed
    if cv2.waitKey(int(1000 / fps)) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()